In [1]:
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import umap
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler

In [2]:
train_df = pd.read_csv("data/train.csv", index_col="id")
test_df = pd.read_csv("data/test.csv", index_col="id")

In [3]:
# Fix some mislabeled days
train_df['expected_day'] = (train_df.index) % 365 + 1
train_df['day_mislabeled'] = train_df['day'] != train_df['expected_day']
train_df.loc[train_df['day_mislabeled'], "day"] = train_df.loc[train_df['day_mislabeled'], "expected_day"]

train_df = train_df.drop(columns=["day_mislabeled", "expected_day"])

In [4]:
# Create artificial years for time series analysis
train_df["year"] = train_df.index // 365
train_df["year"] += 1
test_df["year"] = test_df.index // 365
test_df["year"] += 1

In [5]:
# Fix the one test row with a nan value in winddirection 
test_df.loc[test_df.isna().any(axis=1), "winddirection"] = np.median(train_df[train_df["day"] == 153]["winddirection"])

In [6]:
train_df["date"] = pd.to_datetime((train_df["year"] + 1970).astype(str)) + pd.to_timedelta(train_df["day"] - 1, unit="D")
test_df["date"] = pd.to_datetime((test_df["year"] + 1970).astype(str)) + pd.to_timedelta(test_df["day"] - 1, unit="D")

In [8]:
drop_cols = ["day", "year"]
time_col = "date"

prediction_len = 30